# Cloud Computing Assignment 1
## MSc Business Analytics - Group A8 - A.Y. 2022/2023
### Haifa Abdullah M Alghamdi, Maximilian Bremer, Swan Htet, Alejandro Luque, Giuliano Oscar Stefanelli


<div class="alert alert-warning"> Preliminary step: lease make sure to have these libraries installed and imported</div>

First, we need to import the libraries. The libraries used in this project are:
- *requests*: used to make the actual request to the API.
- *datetime*: used to create a time stamp. 
- *hashlib*: used to create the hash parameter.
- *pandas*: used to create the final dataframe. 
- *flask, flask_restful, flask_bcrypt,* and *flask_jwt_extended*: to successfully create our own API

In [ ]:
#import libraries:
import requests 
import datetime 
import pandas as pd
import hashlib

#if needed, please install the libraries below:
#pip install flask
#pip install flask_restful
#pip install flask_bcrypt
#pip install flask_jwt_extended

## Part 1: The Marvel API

Once registered on the Marvel API website, we will be granted two keys (a public and a private one). Thus, it is a API with a Key Authentication method. 
Even though we should not share the private key, for educational purposes we saved both keys in the variables below.

In [ ]:
public_key = '78250442652c008eed20664b4d2b9b9e'
private_key = '14d25b635eb90f60a4ea54a898d90e66c6114ae9'

As stated on the Marvel API website, server-side applications need two additional parameters: 
- a timestamp: *ts*.
- a md5 digest of the *ts* parameter, your private key and your public key: *hash*. 

> NOTE: The hash parameter is a concatenation of strings, thus we need to convert the timestamp into a string.

In [ ]:
#let's create a timestamp:
ts = datetime.datetime.now()
#convert ts into a string:
ts = ts.strftime("%d/%m/%Y%H:%M:%S")

#let's concatenate the variables and create the hash:
hash_parameter = ts+private_key+public_key
hash_encoded = hashlib.md5(hash_parameter.encode('utf-8')).hexdigest()

Let's verify whether we correctly created the two parameters by making a test call to the Marvel API. The URLs are provided on the website.

To make a request, we first need to create the body of our request (the dictionary *params*) to which we pass the timestamp, the hash, and the public key.

By using the library *requests* and the *get* method, we can make the test call. If everything is correct, we expect a status code of **200**. 

In [ ]:
#let's test if we have access to the Marvel API now:
url_Marvel = 'https://gateway.marvel.com:443/v1/public/characters'

#let's create the body:
params={'apikey': public_key,
       'ts': ts,
       'hash': hash_encoded}

response = requests.get(url_Marvel, params=params)
response.status_code #200 ==> we have correctly accessed the Marvel API

<div class="alert alert-block alert-info"><b>Q1:</b> Provide a list of 30 Marvel characters.</div>

We already have the correct URL to retrieve information about the characters saved in the variable *url_Marvel*. There are no other mandatory values for the body, but we need to specify that we want only 30 characters by setting *limit* to number of wanted characters.

By looking at the structure of the response, we can notice how the wanted information is contained in a nested dictionary. Thus, we will modify the response to fit our needs.

In [ ]:
params['limit'] = 30

characters = requests.get(url_Marvel, params=params)
characters.status_code #200 

#notice how the data we really need is 
characters_30 = characters.json()['data']['results']
characters_30


Let's now create a list with only the names of the 30 Marvel characters in our response. 

<code style="background:yellow;color:black">The answer to Q1 is the list *characters_ls*.</code>

In [ ]:
#let's now create a list:
characters_ls = []

for dictionary in characters_30: 
    characters_ls.append(dictionary['name'])
    
characters_ls

<div class="alert alert-block alert-info"><b>Q2:</b> Retrieve the IDs for all the characters in your list (in str form).</div>

To complete Q2, we need to access *characters_30*.

For visual ease, we decided to store the IDs of the characters in a dictionary which keys are the names of the characters and the values are their respective IDs.

<code style="background:yellow;color:black">The answer to Q2 is stored in the dictionary *characters_IDs*.</code>

In [ ]:
IDs = []
for i in characters_30:
    IDs.append(i['id'])

i = 0
charcters_IDs = {}

while i < len(IDs):
    charcters_IDs[characters_ls[i]] = str(IDs[i]) #remember we were asked to store the IDs in string format!
    i += 1
    
charcters_IDs

<div class="alert alert-block alert-info"><b>Q3:</b> Retrieve the total number of Events available for all the characters in your list (in integer form).</div>

<div class="alert alert-block alert-info"><b>Q4:</b> Retrieve the total number of Series available for all the characters in your list (in integer form).</div>

<div class="alert alert-block alert-info"><b>Q5:</b> Retrieve the total number of Comics available for all the characters in your list (in integer form).</div>

These three questions can be individually solved using the same approach by changing the parameter passed (events, series, or comics). For this reason, we decided to use a function so that we just need to change the parameter *elem*. 

<code style="background:yellow;color:black">The answer to Q3 is stored in the dictionary *characters_events*.</code>

<code style="background:yellow;color:black">The answer to Q4 is stored in the dictionary *characters_series*.</code>

<code style="background:yellow;color:black">The answer to Q5 is stored in the dictionary *characters_comics*.</code>



In [ ]:
def get_total_number_of(elem):
    ls = []
    for i in characters_30:
        ls.append(i[elem]['available']) 
        
    x = 0
    result = {}
    while x < len(ls):
        result[characters_ls[x]] = int(ls[x])  #remember we were asked to store the total number in int format!
        x += 1
    return result

In [ ]:
characters_events = get_total_number_of('events')
characters_events

In [ ]:
characters_series = get_total_number_of('series')
characters_series

In [ ]:
characters_comics = get_total_number_of('comics')
characters_comics

<div class="alert alert-block alert-info"><b>Q6:</b> Retrieve the Price of the most expensive comic that the character was featured in or all the characters in your list (in float form and USD).</div>

With the request we have previously made to the Marvel API, we do not have access to the price of the comics. Thus, we need to make another call to the API. 

There is a special case for the character Adam Warlock who has appeared in 188 comics. Since the Marvel API only gathers up to the first 100 entries, the additional offset needs to be taken into consideration.

The Marvel API website states that the timestamp can change on a request-by-request basis, thus we decided to create a new one. 

In [ ]:
#let's create a new timestamp:
ts_1 = datetime.datetime.now()

#since we have to concatenate the timestamp to the public and private key, we should convert ts into a string.
ts_1 = ts_1.strftime("%d/%m/%Y%H:%M:%S")

#let's create the hash:
hash_parameter_1 = ts_1+private_key+public_key
hash_encoded_1 = hashlib.md5(hash_parameter_1.encode('utf-8')).hexdigest()

In [ ]:
comics_info = {}

for char in charcters_IDs:
    url_Marvel_1 = f'https://gateway.marvel.com:443/v1/public/characters/{charcters_IDs[char]}/comics'
    params={'apikey': public_key,
       'ts': ts_1,
       'hash': hash_encoded_1,
       'characterId' : charcters_IDs[char],
       'limit':100
       }
    comics_info[char] = (requests.get(url_Marvel_1, params=params)).json()['data']['results']
    if charcters_IDs[char] == '1010354':
        url_Marvel_2 = f'https://gateway.marvel.com:443/v1/public/characters/1010354/comics'
        params2={'apikey': public_key,
           'ts': ts_1,
           'hash': hash_encoded_1,
           'characterId' :'1010354',
           'limit':100,
           'offset':100
           }
        extra_88 = (requests.get(url_Marvel_2, params=params2)).json()['data']['results']
        comics_info['Adam Warlock'] += extra_88


comics_info

Let's now find a way to retrieve the most expensive comic for each character. We will store this information in the dictionary *character_comic_price*.

We noticed that *character_comic_price* does not contain those characters who have not appeared in any comics. Hence we create a new dictionary *character_comic_price_final* to come up with the missing couple character-max price.
 

In [ ]:
character_comic_price = {}
for characters in comics_info: 
    character_comics_info = comics_info[characters]
    price = []
    for comic_id in character_comics_info: 
        for comic_price in comic_id['prices']: 
            price.append(float(comic_price['price'])) #remember we were asked to store the price in float format!
        character_comic_price[characters] = (max(price))

character_present = [] # characters with comics and highest relative price
for character in character_comic_price: 
    character_present.append(character)       

character_comic_price_final = {}

for character in comics_info:
    if character in character_present: 
        character_comic_price_final[character] = character_comic_price[character]
    else: 
        character_comic_price_final[character] = float(0) #remember we were asked to store the price in float format!

character_comic_price_final


<div class="alert alert-block alert-info"><b>Q7:</b> Store the data above in a pandas DataFrame called df containing exactly in the following columns: Character Name, Character ID, Total Available Events, Total Available Series, Total Available Comics, Price of the Most Expensive Comic.</div>

> If a character is not featured in Events, Series or Comics the corresponding entry should be filled in with a None (of NoneType). If a character does not have a Price the corresponding entry should be filled in with a None (of NoneType).


Remember that the Character ID should be of string type!

In [ ]:

df = pd.DataFrame()
df['Character Name'] = characters_ls
df['Character ID']= charcters_IDs.values()
df['Total Available Events'] = characters_events.values()
df['Total Available Series'] = characters_series.values()
df['Total Available Comics'] = characters_comics.values()
df['Price of the Most Expensive Comic'] = character_comic_price_final.values()

df.replace(0, None, inplace=True)
df = df.astype({
    'Character ID' : 'str'})


<div class="alert alert-block alert-info"><b>Q8:</b> Save df to a file called data.csv.</div>

In [ ]:
df.to_csv('data.csv', index=False)

The CSV file consists in 6 columns of 30 observations, namely:
1. Character Name: the name of the Marvel character.
2. Character ID: unique identifier of the Marvel character.
3. Total Available Events: the number of events the character has appeared in.
4. Total Available Series: the number of series the character has appeared in.
5. Total Available Comics: the number of comics the character has appeared in.
6. Price of the Most Expensive Comic: the most expensive comic in which the character has appeared in (in US dollars).

## Part 2: The Marvel x Group A8 API

For Part 2, the report will be structured in the following three steps:

1. In the first section, we will comment the file '*api_release_a8.py*' which contains the code used to create what we call *The Marvel x Group A8 API*. We will keep the description of the different classes and methods short.

2. As all APIs do, in the second section, we will provide a documentation on how to use *The Marvel x Group A8 API* and its functionalities. 

3. In the third section, we will provide you with some sample codes that you can use to perform some requests to our API.



## Section 1: 

<div class="alert alert-block alert-info">
<b>Q1:</b> Create an API that allows users to interact with the DataFrame generated in the Part 1 of the assignments.
    
<b>Q2:</b> Create a resource called Characters and route it to the url '/characters' and endpoint 'characters'.
</div>

<div class="alert alert-block alert-info">
<b>Q3-1:</b> Implement the method for the Character resource to retrieve the whole DataFrame in json format.
    
<b>Q3-2:</b> Implement the method for the Character resource to retrieve information for a single entry or for a list of entries identified by either the Character Name or the Character ID.
</div>

We created the *get* method by tackling the above questions together. Indeed, the final user will be able to:
- retrieve the entire dataframe if she/he will not specify the name and/or ID of the character.
- retrieve part of the dataframe based on her/his request. The user can input a single ID, a single name, a list of IDs, or a list of names considering all the combinations of these input. For example, the user can input a list of IDs and a single name in the same *get* request.

<div class="alert alert-block alert-info">
<b>Q3-3:</b> Implement the method for the Character resource to add a new character to the existing DataFrame by specifying its characteristics (Character Name, Character ID, Available Events, Available Series, Available Comics, and Price of Comic). The API should restrict addition of characters with pre-existing Character IDs.
    
<b>Q3-4:</b> Implement the method for the Character resource to add a new character to the existing DataFrame by specifying only the Character ID. The API should fill in the remaining information by extracting it from Marvel's API and appending to the DataFrame. The API should return an error if the provided Character ID is not found.
</div>

We created the *post* method by tackling the above questions together. To add a new character to the existing dataframe, however, the user will need to provide her/his access token in the header of the request (see samples later). The final client will have two options:
1. She/he can provide all the necessary information and the new character will be added automatically (if already not present in the dataframe).
2. She/he can provide only the ID of the character wanted and the other information will be retrieved from the full Marvel API.

<div class="alert alert-block alert-info"><b>Q3-5:</b> Implement the method for the Character resource to Delete a character or a list of characters by providing either the Character ID or the Character Name. The API should return an error if the character you are trying to delete does not exist in the DataFrame.</div>

To delete a character from the existing dataframe, the user once again needs to privide an access token. After that, the user can ask to delete a single character or a list of characters by provided the ID and/or name. For example, the client can ask to delete multiple characters by providing a list of their IDs and, in the same request, delete a single character by providing its name.


<div class="alert alert-block alert-info"><b>Q4:</b> Protect both the addition and the deletion of characters using an OAuth authentication scheme whereby users can sign up and then log in to obtain an access token with limited scope and a duration of 1 hour.</div>


To protect the *post* and *delete* methods, we created:
- a class SignUp that will allow the user to register before using our API.
- a class LogIn that will allow the users to login into our API and gain access to the token.


<div class="alert alert-block alert-info"><b>Bonus:</b> Write the code to enable users to modify the Price of the Most Expensive Comic by providing either the Character ID or the Character Name. The API should accept new prices in different currencies, including USD, EUR, GBP and CAD and transform them to the right values to the exanche rate of the considered date and time (+/- an hour).</div>

The user will be able to update the entries for the column 'Price of the Most Expensive Comic' by converting the currencies. Remember: the original currency is USD.
> The user can convert to/from USD, EUR, GBP, CAN!

## Section 2: 

### SIGN-UP METHOD:

How to sign up to our API.


URL: 'http://localhost:5000/signup'
To structure the sign-up requests:


To structure the sign-up requests:

| Parameters | Data Type | Description |
| --- | --- | --- | 
| email (required)| str | User's email | 
| password required)| str | Create your own password| 

Errors Status Codes:

| HTTP Status Code | Reason| 
| --- | --- |
| 409 | email already exists in the database | 


### LOGIN METHOD:

How to login to our API.


URL: 'http://localhost:5000/login

To structure the login requests:

| Parameters | Data Type | Description |
| --- | --- | --- | 
| email (required)| str | User's email | 
| password required)| str | User's password| 


REMEMEBER: after you login, you will be able to get the access_token needed to execute post and delete requests.

(see Section 3 for sample code).

Errors Status Codes:

| HTTP Status Code | Reason| 
| --- | --- |
| 401 | email not present in the database | 
| 402 | password not valid | 


### GET METHOD:

Fetches a single character or a list of characters.


URL: 'http://localhost:5000/characters'


To structure the get requests:

| Parameters | Data Type | Description | Examples |
| --- | --- | --- | --- |
| characterId (not required) | str | The ID (or list of IDs) of the character(s) you want to retrieve | '1011334' or ['1011334', '1010354'] |
| characterName (not required)| str |The name (or list of names) of the character(s) you want to retrieve| '3-D Man' or ['3-D Man', 'A-Bomb'] |

Errors Status Codes:

| HTTP Status Code | Reason| 
| --- | --- |
| 404 | Character Name or Character ID requested is not present in our dataframe | 


### POST METHOD:

Adds a character to the existing dataframe.


URL: 'http://localhost:5000/characters'


To structure the post requests:

| Parameters | Data Type | Description | Examples |
| --- | --- | --- | --- |
| Authorization (required) | str | It is the required access token given to the user | put example |
| characterId (required) | str | The ID of the character you want to add | '1011334' |
| characterName (not required)| str |The name of the character you want to retrieve| '3-D Man'|
| number_events (not required)| int |The number of events that the character has appeared in | 1|
| number_series (not required)| int |The number of series that the character has appeared in | 2|
| number_comics (not required)| float |The number of comics that the character has appeared in | 3|

Errors Status Codes:

| HTTP Status Code | Reason| 
| --- | --- |
| 401 | Connection not established | 
| 402 | Insufficient information | 
| 404 | Character ID not found | 
| 409 | Character ID already present in the dataframe| 


### DELETE METHOD:

Deletes a character or a list of characters from the existing dataframe.


URL: 'http://localhost:5000/characters'


To structure the delete requests:

| Parameters | Data Type | Description | Examples |
| --- | --- | --- | --- |
| Authorization (required) | str | It is the required access token given to the user | put example |
| characterId (not required) | str | The ID of the character you want to delete | '1011334' |
| characterName (not required)| str |The name of the character you want to delete| '3-D Man'|

Errors Status Codes:

| HTTP Status Code | Reason| 
| --- | --- | 
| 404 | characterId and/or characterName not specified | 
| 405 | characterId and/or characterName input not present in the dataframe| 

### PUT METHOD:

Updates the highest price of the comic by converting the price from USD to the currency chosen by the user.


URL: 'http://localhost:5000/characters' 


To structure the delete requests:

| Parameters | Data Type | Description | Examples |
| --- | --- | --- | --- |
| Authorization (required) | str | It is the required access token given to the user | put example |
| characterId | str | The ID of the character you want to delete | '1011334' |
| characterName | str |The name of the character you want to delete| '3-D Man'|
| original_currency (required)| str |The original currency (initially is 'USD')| 'USD'|
| wanted_currency (required)| str |The wanted currency (accepts: 'USD', 'EUR', 'GBP', 'CAD'| 'EUR'|

Errors Status Codes:

| HTTP Status Code | Reason| 
| --- | --- | 
| 405 | characterId/characterName input not found| 

## Section 3: 

In [ ]:
# build dataframe to store users information
df = pd.DataFrame(columns=['email', 'password'])
df.to_csv('users.csv', index=False)

## Welcome to *The Marvel x Group A8 API*
### Please register to use our API:

In [ ]:
# Sign-up process:
requests.post('http://localhost:5000/signup', params={'email': 'irene.unceta@esade.edu', 'password': '0000'}).json()

### Thank your for registering, please login to start using the API:

In [ ]:
# Login process:
response = requests.get('http://localhost:5000/login', params={'email': 'irene.unceta@esade.edu', 'password': '0000'}).json()
response

### You're logged in! 
#### You can now start making requests to our API, remember to use your access token to add or delete characters. 

In [ ]:
access_token = response['token'] #rememeber: your token expires after 1 hour!

### "Get" request without specifying which character(s) to retrieve:

In [ ]:
response = requests.get('http://localhost:5000/characters')
response.status_code

response.json()['response']



### "Get" request specifying which character(s) to retrieve:
#### Remember: you can provide a single ID/a list of IDs and/or a single name/a list of names.

In [ ]:
#requests.get('http://localhost:5000/characters', params={'characterName': ['Adam Destine','Adam Warlock'] , 'characterId':['1010903','1009149']}).json()['response']
requests.get('http://localhost:5000/characters', params={'characterId':['1010903','1009149']}).json()['response']



### "Post" request specifying all the necessary information of the new character:


In [ ]:
requests.post('http://localhost:5000/characters', 
              params={
                  'characterName': 'CloudComputing',
                  'characterId': '12345',
                  'number_events': 1,
                  'number_series': 2,
                  'number_comics': 3,
                  'highest_price': 123
              },
              headers={'Authorization': f'Bearer {access_token}'}).json()


In [ ]:
# check if the data set has been updated correctly:
data = pd.read_csv('data.csv')
data 

### "Post" request specifying only the character ID: 

In [ ]:
requests.post('http://localhost:5000/characters', 
              params={
                  'characterId': '1009435'
              },
              headers={'Authorization': f'Bearer {access_token}'}).json()

In [ ]:
# check if the data set has been updated correctly:
data = pd.read_csv('data.csv')
data

### "Delete" request specifying only the character ID (character name): 
#### Remember: you can provide a single ID/a list of IDs and/or a single name/a list of names.

In [ ]:
requests.delete('http://localhost:5000/characters', params={'characterId': 1009435},headers={'Authorization': f'Bearer {access_token}'}).json()


In [ ]:
# check if the data set has been updated correctly:
data = pd.read_csv('data.csv')
data

### "Delete" request specifying only the character ID (character name): 

In [ ]:
requests.put('http://localhost:5000/characters', params = {
    'original_currency': 'USD',
    'wanted_currency': 'GBP',
    'characterId':'12345'},headers={'Authorization': f'Bearer {access_token}'}).json()

In [ ]:
# check if the data set has been updated correctly:
data = pd.read_csv('data.csv')
data